# Importação dos dados

In [3]:
# Para manipulaçao dos dados
import pandas as pd

In [4]:
url = 'https://github.com/fellypesb/time_series_predict/raw/main/Dados/local_BR/train_local_BR.csv'

local_BR_train = pd.read_csv(url)

# Células notebook antigo

In [ ]:
# Divisão dos dados entre variáveis preditoras e alvos

atributos_ufopa = ufopa_[best_corr.index]

target_ufopa = ufopa_['ghi']

(atributos_ufopa.shape, target_ufopa.shape,)

In [ ]:
# boxplot dos atributos previsores

with sns.axes_style('whitegrid'):
    fig, ax = plt.subplots(figsize=(15,7))
    sns.boxplot(data=atributos_ufopa, palette='Set2')
    ax.set_title('Distribuição dos dados', fontdict={'fontsize':18, 'fontweight': 'bold'})
    plt.xticks([0,1,2,3],['angulo_zenital', 'irradiancia_ceu_claro', 'umidade_relativa', \
    'temperatura'])
    plt.show()

### todo
+ Mudar título
+ Adicionar labels
+ Entender o que seria gráfico distorcido
+ Entender possíveis outlinears

In [ ]:
# Divisão dos dados para treino e teste (cronológica)

X_train, X_test, y_train, y_test = train_test_split(atributos_ufopa, target_ufopa, test_size=0.2, shuffle=False)



### todo
+ Buscar mais informações sobre divisão dos dados: **treino/validação/teste**
+ fazer plot com o seaborn
+ Melhor forma de definir o tamanho de treino e teste é pensar no produto final. Qual é o horizonte de previsões adequado para melhor solucionar o problema?
+ Equilibrio entre a robustez da estimativa de validação e a capacidade do modelo de aprender com uma quantidade mínima necessária de exemplos
+ Fazer uma análise da pontação do modelo para diferentes tamanhos de conjuntos de treino e teste  

In [ ]:
print('Treino:', X_train.shape[0], '\tTeste:', X_test.shape[0])


In [ ]:
# Calcula o horizonte de previsões

q_dias = X_test.shape[0] / 13
q_meses = q_dias / 30
print(f'Horizonte de previsões aprox. {round(q_meses,2)} meses.')

# Modelos Dummy

São modelos que realizam previsões utilizando regras simples.

Servem como baseline para comparar com outros modelos. Não é utilizado em problemas reais.

### Média do atributo alvo

In [ ]:
dummy_1 = pd.DataFrame({'y_real':y_test, 'y_pred':y_train.mean()})
accuracy_1 = r2_score(dummy_1.y_real, dummy_1.y_pred)

with sns.axes_style('whitegrid'):
    plt.figure(figsize=(20,8))
    plt.plot(y_test, label='Real')
    plt.plot(dummy_1.y_pred, label='Pred.')
    plt.xlabel('Tempo (h)', fontsize=20)
    plt.ylabel('Irradiância ($W/m^2$)', fontsize=20)
    plt.title(f'Score: {accuracy_1}', fontsize=20, weight='bold')
    plt.legend()

## Coeficiente de determinação ($R^2$ )
# <center>$R^2(y, \hat{y}) = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}$</center>

Onde $y_i$ corresponde ao valor esperado, $\hat{y}_i$ o valor predito para o i-ésimo exemplo e $\bar{y}$ a média do alvo das predições. Nesta métrica, o melhor valor que pode ser encontrado é 1 (um).

### Médias dos atributos previsores

In [ ]:
dummy_2 = pd.DataFrame({'y_real':y_test, 'y_pred':X_test.std(axis=1)})
accuracy_2 = r2_score(dummy_2.y_real, dummy_2.y_pred)

with sns.axes_style('whitegrid'):
    plt.figure(figsize=(20,8))
    dummy_2['y_real'].plot(label='Real')
    dummy_2['y_pred'].plot(marker='.', label='Pred.')
    plt.xlabel('Tempo (h)', fontsize=20)
    plt.ylabel('Irradiância ($W/m^2$)', fontsize=20)
    plt.title(f'Score: {round(accuracy_2,2)}', fontsize=20, weight='bold')
    plt.legend()

### Lags (atrasos de tempo)

In [ ]:
dummy_3 = pd.DataFrame({'y_real':y_test, 'y_pred':y_test.shift(13)}).dropna()
#accuracy_3 = r2_score(dummy_3.y_real, dummy_3.y_pred)
accuracy_3 = mean_absolute_percentage_error(dummy_3.y_real, dummy_3.y_pred)
#accuracy_3 = mean_absolute_error(dummy_3.y_real, dummy_3.y_pred)

with sns.axes_style('whitegrid'):
    dummy_3['y_real'].plot(figsize=(20,8), label='Real')
    dummy_3['y_pred'].plot(label='Pred.')
    plt.xlabel('Tempo (h)', fontsize=20)
    plt.ylabel('Irradiância ($W/m^2$)', fontsize=20)
    plt.title(f'Score: {round(accuracy_3,2)}', fontsize=20, weight='bold')
    plt.legend()

### todo
+ estudar e plotar gráfico de autocorreação para melhor entendimento do baseline anterior

### Médias Móveis

In [ ]:
dummy_4 = pd.DataFrame({'y_real':y_test, 'y_pred':y_test.rolling(2).mean()}).dropna()
#accuracy_4 = r2_score(dummy_4.y_real, dummy_4.y_pred)
#accuracy_4 = mean_absolute_percentage_error(dummy_3.y_real, dummy_3.y_pred)
accuracy_4 = mean_absolute_error(dummy_3.y_real, dummy_3.y_pred)

with sns.axes_style('whitegrid'):
    dummy_4['y_real'].plot(figsize=(20,8), label='Real')
    dummy_4['y_pred'].plot(label='Pred.')
    plt.xlabel('Tempo (h)', fontsize=20)
    plt.ylabel('Irradiância ($W/m^2$)', fontsize=20)
    plt.title(f'Score: {round(accuracy_4,2)}', fontsize=20, weight='bold')
    plt.legend()